In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
import numpy as np
np.random.seed(2016)

import os
import glob
import cv2
import datetime
import pandas as pd
import time
import warnings
warnings.filterwarnings("ignore")

from sklearn.cross_validation import KFold
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping

from sklearn.metrics import log_loss
from keras import __version__ as keras_version
%matplotlib inline
from __future__ import division,print_function
import os, json
from glob import glob
import numpy as np
import scipy
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix
np.set_printoptions(precision=4, linewidth=100)
from matplotlib import pyplot as plt


from numpy.random import random, permutation
from scipy import misc, ndimage
from scipy.ndimage.interpolation import zoom

import keras
from keras import backend as K

from keras.models import Sequential
from keras.layers import Input
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD, RMSprop
from keras.preprocessing import image

In [ ]:
X_train = []
X_train_id = []
y_train = []
tart_time = time.time()
print('Read train images')
folders = ['Type_1', 'Type_2', 'Type_3']
for fld in folders:
    index = folders.index(fld)
    print (index)
    print('Load folder {} (Index: {})'.format(fld, index))
    path = os.path.join('..', 'input', 'train', fld, '*.jpg')
    files = glob(path)
    for o,fl in enumerate(files):
        flbase = os.path.basename(fl)
        img = cv2.imread(fl)
        #print (type(img))
        img = cv2.resize(img, (32,32))
        X_train.append(img)
        X_train_id.append(flbase)
        y_train.append(index)

In [ ]:
print (len(y_train),len(X_train)),len(X_train_id)

In [ ]:
path = os.path.join('..', 'input', 'test', '*.jpg')
files = sorted(glob(path))
X_test = []
X_test_id = []
for fl in files:
    flbase = os.path.basename(fl)
    img = cv2.imread(fl)
    #print (type(img))
    img = cv2.resize(img, (32,32))
    X_test.append(img)
    X_test_id.append(flbase)

In [ ]:
def create_model():
    model = Sequential()
    model.add(ZeroPadding2D((1, 1), input_shape=(3, 32, 32), dim_ordering='th'))
    model.add(Convolution2D(4, 3, 3, activation='relu', dim_ordering='th'))
    model.add(ZeroPadding2D((1, 1), dim_ordering='th'))
    model.add(Convolution2D(4, 3, 3, activation='relu', dim_ordering='th'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), dim_ordering='th'))
    model.add(ZeroPadding2D((1, 1), dim_ordering='th'))
    model.add(Convolution2D(8, 3, 3, activation='relu', dim_ordering='th'))
    model.add(ZeroPadding2D((1, 1), dim_ordering='th'))
    model.add(Convolution2D(8, 3, 3, activation='relu', dim_ordering='th'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), dim_ordering='th'))
    model.add(Flatten())
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(3, activation='softmax'))
    
    return model

In [ ]:
"""
def create_model():
    model = Sequential()
    
    model.add(ZeroPadding2D((1, 1), input_shape=( 3, 32, 32), dim_ordering='th'))
    
    model.add(Convolution2D( 128, 3, 3, activation = 'relu', dim_ordering='th') )
    
    model.add(ZeroPadding2D((1, 1),  dim_ordering='th'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), dim_ordering='th'))
                           
    model.add(ZeroPadding2D((1, 1), dim_ordering='th'))
    model.add(Convolution2D( 64, 3, 3, activation = 'relu', dim_ordering='th') )
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), dim_ordering='th'))
    
    model.add(ZeroPadding2D((1, 1), dim_ordering='th'))
    model.add(Convolution2D( 128, 3, 3, activation = 'relu', dim_ordering='th') )
  
    
    
    model.add(Flatten())
    
    model.add(Dense(32, activation='relu'))
   
    model.add(Dropout(0.5))
    model.add(BatchNormalization())
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.5))
    model.add(BatchNormalization())
    model.add(Dense(3, activation='softmax'))
              
    return model
    
    """

In [ ]:
from keras.layers.normalization import BatchNormalization
model = create_model()

In [ ]:
len(y_train)

In [ ]:
from keras.utils.np_utils import to_categorical
def onehot(x):
    return to_categorical(x,3)

batch_size = 64
X_train = np.array(X_train).transpose((0, 3, 1, 2))
X_train = X_train / 255.0
from sklearn.cross_validation import train_test_split
X_train1, X_valid, Y_train, Y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=42)


Y_train = onehot(Y_train)
Y_valid = onehot(Y_valid)
nb_epoch = 10
#opt = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
#model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
sgd = SGD(lr=0.01)
model.compile(optimizer=sgd, loss='categorical_crossentropy',
    metrics=['accuracy'])
model.fit(X_train1, Y_train, batch_size=batch_size, nb_epoch=16,
              shuffle=True, verbose=2, validation_data=(X_valid, Y_valid))

In [ ]:
X_test = np.array(X_test).transpose((0, 3, 1, 2))
X_test = X_test / 255.0
prediction = model.predict(X_test)
k=pd.DataFrame({'image_name':X_test_id,'Type_1':prediction[:,0],'Type_2':prediction[:,1],'Type_3':prediction[:,2]})


In [ ]:
k=pd.DataFrame({'image_name':X_test_id,'Type_1':prediction[:,0],'Type_2':prediction[:,1],'Type_3':prediction[:,2]})

In [ ]:
k

In [ ]:
k.to_csv('sub1.csv',index = False)

In [ ]:
from IPython.display import FileLink
FileLink('sub1.csv')